In [28]:
from Elgamal import Elgamal, ElgamalBig
import time
import tracemalloc

In [29]:
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time

In [30]:
def measure_time_decode(func, enc_text, private_key, public_key):
    start_time = time.time()
    result = func(enc_text, private_key, public_key)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time

In [31]:

with open("zapovit.txt", 'r', encoding="utf-8") as file:
    small_text = file.read()
with open("kateryna.txt", 'r', encoding="utf-8") as file:
    medium_text = file.read()
with open("tygrolovy.txt", 'r', encoding="utf-8") as file:
    big_text = file.read()



el = ElgamalBig()
tracemalloc.start()
(public_key, private_key), time = measure_time(el.generate_keys)
memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)
tracemalloc.clear_traces()
print(public_key, private_key)
print(f'The time used to generate keys: {time}s')
print(f'The memory used to create keys: {memory_usage} MB')


(29657099953955114013088582898737391720462265288034579975530845521445661768411085195837037976997505637371685754482455938807184053209070380022042304993129609223641821112889077790567044561771668689877424020869211054656204074756229928219225482168818209410012615502960925613344341246507781842851611835713646734850700880129102733495351800229725622804920136273024450766185661420131475210430463825183216869993650934991633148630256811050235321437665971616455870246018664384578516951706867433173652106206110585553566655520058197923008188424682115649813648883738183683853106065860155342964838031634609146882907742485165906751123, 13084487121856860641552073579445652803244407652295056188264409216991649156958928005772028345342857410372620009988415628725656174199327280219631419592744117, 5153253381136295489510408541566119134827744031665215631953355728135211222672644935243081855918995213614956914660266357464710335545805737842245211882248886130749921843564020479297353468996172934738758112169779683192625939848

Результати тестування еіективності алгоритму на текстових файлах різних розмірів:

In [15]:
for text1 in [(small_text, 'small'), (medium_text, 'medium'), (big_text, 'big')]:
    text, name = text1
    tracemalloc.start()
    encoded_result, encoding_time = measure_time(el.encrypt, text, public_key)
    encoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    tracemalloc.start()
    decoded_result, decoding_time = measure_time_decode(el.decrypt, encoded_result, private_key, public_key)
    decoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    print(f"Correctness: {text == decoded_result}")
    print(f"The time used for encoding {name} text is {encoding_time}, memory - {encoding_memory_usage} MB")
    print(f"The time used for decoding {name} text is {decoding_time}, memory - {decoding_memory_usage} MB")

Correctness: True
The time used for encoding small text is 0.5002198219299316, memory - 2.650509834289551 MB
The time used for decoding small text is 0.04553699493408203, memory - 0.007800102233886719 MB
Correctness: True
The time used for encoding medium text is 1.5568153858184814, memory - 0.04699516296386719 MB
The time used for decoding medium text is 1.3722612857818604, memory - 0.07665824890136719 MB
Cannot decode the message. The encrypted message may have been tampered with.
Correctness: False
The time used for encoding big text is 40.58422350883484, memory - 1.2032785415649414 MB
The time used for decoding big text is 36.977336168289185, memory - 1.197866439819336 MB


TypeError: write() argument must be str, not None

З дуже великими даними цей алгоритм не справляється.

In [16]:
from encode_photo import encode_image_to_base64
from decode_photo import decode_base64_file_to_image

small_photo_encoded = encode_image_to_base64('very_small_image.jpeg')
large_photo_encoded = encode_image_to_base64('photo.jpg')

Тестування ефективності алгоритму на картинках (без урахування часу на перетворення самих фото):

In [20]:
for file, name in [(small_photo_encoded, 'small'), (large_photo_encoded, 'large')]:
    tracemalloc.start()
    encoded_result, encoding_time = measure_time(el.encrypt, file, public_key)
    encoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    tracemalloc.start()
    decoded_result, decoding_time = measure_time_decode(el.decrypt, encoded_result, private_key, public_key)
    decoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    print(f"Correctness: {file == decoded_result}")
    print(f"The time used for encoding {name} photo is {encoding_time}, memory - {encoding_memory_usage} MB")
    print(f"The time used for decoding {name} photo is {decoding_time}, memory - {decoding_memory_usage} MB")

Correctness: True
The time used for encoding small photo is 4.339330673217773, memory - 0.1809101104736328 MB
The time used for decoding small photo is 3.716301202774048, memory - 0.11814308166503906 MB
Correctness: True
The time used for encoding large photo is 12.718733549118042, memory - 0.35483551025390625 MB
The time used for decoding large photo is 11.476421117782593, memory - 0.35382938385009766 MB
